In [1]:
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2
from openproblems.tasks.regulatory_effect_prediction import datasets, methods


In [63]:
# test = False does not work as tss_to_peaks need to be sub-sampled, and it seems that everything is blended.
adata = datasets.snare_p0_braincortex(test=False)

In [64]:
# the genes will only be limited to chromosome 1`(faster calculation, no randomness)

In [65]:
# this indicates the number of genes we will test simultaneously in our 
n_genes = 2000

In [76]:
adata_sample = adata[:,adata.var.index.isin(adata.var.head(n_genes).index)]

In [77]:
adata_sample = methods.rp_simple(adata_sample, n_top_genes=2000) #  log_each=10)

... storing 'chr' as categorical
... storing 'strand' as categorical
/home/icb/ignacio.ibarra/miniconda3/envs/openproblems/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


# weights calculated so far 0 out of 1452
# weights calculated so far 500 out of 1452
# weights calculated so far 1000 out of 1452


/mnt/znas/icb_zstore01/groups/ml01/workspace/ignacio.ibarra/SingleCellOpenProblems/openproblems/tasks/regulatory_effect_prediction/methods/maestro.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shape=(adata.shape[1], adata.uns["mode2_var"].shape[0]),


In [78]:
%autoreload 2

import seaborn as sns
from openproblems.tasks.regulatory_effect_prediction import metrics

cors = metrics.spearman_correlation(adata_sample), metrics.pearson_correlation(adata_sample)
print(cors)

(-0.017173285122470874, -0.011103724554606167)


In [79]:
adata_sample = adata[:,adata.var.index.isin(adata.var.head(n_genes).index)]

In [80]:
adata_sample = methods.rp_enhanced(adata_sample, n_top_genes=2000)

... storing 'chr' as categorical
... storing 'strand' as categorical
/home/icb/ignacio.ibarra/miniconda3/envs/openproblems/lib/python3.7/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


# weights calculated so far 0 out of 1452
# weights calculated so far 500 out of 1452
# weights calculated so far 1000 out of 1452


In [81]:
%autoreload 2

import seaborn as sns
from openproblems.tasks.regulatory_effect_prediction import metrics

cors = metrics.spearman_correlation(adata_sample), metrics.pearson_correlation(adata_sample)
print(cors)

(-0.017543645705293393, -0.011540115273533868)
